Reading sounding data from bufr files

In [2]:
"""
author: Nora Helgeland
date: May, 2023

"""

import pandas as pd
import numpy as np
import cartopy.crs as ccrs
import matplotlib.pyplot as plt
import cartopy
import matplotlib as mpl
#import geopandas
import math
import cartopy as cr
#from mpl_toolkits.basemap import Basemap
from scipy.interpolate import griddata
import matplotlib.cbook as cbook
import cartopy.feature as cfeature
from numpy import nan
#import seaborn as sns
%matplotlib inline
%config InlineBackend.figure_format = 'svg'
plt.rcParams['xtick.direction'] = 'in'
plt.rcParams['ytick.direction'] = 'in'
plt.rcParams['axes.unicode_minus'] = False
plt.rcParams.update({'font.size': 12})
import re
import pandas as pd
import pdbufr
import numpy as np

Load the bufr file for the desired date

In [3]:
df = pdbufr.read_bufr("/lustre/storeB/project/fou/kl/cerad/Meteorology/AROME-CHERNOBYL/RadioSondes/Data/temp1986050512",
columns=("latitude", "longitude", "pressure", "airTemperature", "stationNumber", "windDirection", "windSpeed", "nonCoordinateGeopotential", "heightOfStation"))
grouped = df.groupby(["latitude", "longitude"])
grouped

In [4]:
#Create height column from the geopotential column

df["height"] = df["nonCoordinateGeopotential"]/9.81
df


,stationNumber,latitude,longitude,heightOfStation,pressure,nonCoordinateGeopotential,airTemperature,windDirection,windSpeed,height
0,360.0,65.60,-37.63,52.0,102800.0,510.0,268.9,0.0,0.0,51.987768
1,360.0,65.60,-37.63,52.0,100000.0,2690.0,272.5,310.0,3.0,274.209990
2,360.0,65.60,-37.63,52.0,99500.0,3050.0,273.2,NaN,NaN,310.907238
3,360.0,65.60,-37.63,52.0,98800.0,NaN,NaN,310.0,4.0,NaN
4,360.0,65.60,-37.63,52.0,95500.0,6280.0,275.6,NaN,NaN,640.163099
...,...,...,...,...,...,...,...,...,...,...
28236,328.0,16.42,120.60,1500.0,25000.0,107970.0,232.7,285.0,6.7,11006.116208
28237,328.0,16.42,120.60,1500.0,20000.0,122490.0,221.1,270.0,10.3,12486.238532
28238,328.0,16.42,120.60,1500.0,15000.0,140140.0,206.9,250.0,5.2,14285.423038
28239,328.0,16.42,120.60,1500.0,10000.0,163280.0,191.9,230.0,4.1,16644.240571


Limiting the area to be looked at

In [5]:
lat_mask = df["latitude"].between(48, 80)
long_mask = df["longitude"].between(5, 40)

df_filter = df[lat_mask & long_mask]
            


Placing the atmospheric soundings in csv files

In [6]:

for i, part_df in df_filter.groupby(["latitude", "longitude"]):

    part_df.to_csv(f'/home/norah/master/Rs_data_bufr/1986050512/{round(i[0],2)}_{round(i[1],2)}.csv', index=False)
     

Make a map showing the position of the radiosondes in the bufr file

In [ ]:
df2=df.dropna(subset=["latitude","longitude"])
df3=df.loc[:, ["latitude","longitude"]].drop_duplicates().values 
df3
pairs_df = pd.DataFrame(df_filter, columns=["latitude","longitude"]) 
pairs_df

In [161]:
# %matplotlib widget
#fig,ax = plt.subplots(subplot_kw={'projection':ccrs.PlateCarree()}, figsize=(10,15))

fig,ax = plt.subplots(subplot_kw={'projection':ccrs.LambertConformal(central_longitude=10, standard_parallels=(54,78))}, figsize=(10,15))
ax.scatter(30.1, 51.4, color='blue', transform=ccrs.PlateCarree())
ax.scatter(pairs_df["longitude"], pairs_df["latitude"], color='red', transform=ccrs.PlateCarree())
gl = ax.gridlines(draw_labels=True)
gl.top_labels=False
gl.right_labels=False

#borders,lakes,ocean,coastline,rivers and so on 
provinces_50m = cfeature.NaturalEarthFeature('cultural',
                                             'admin_1_states_provinces_lines',
                                             '50m',
                                             facecolor='none')

#ax.set_extent((-20, 80, 51, 88), crs=ccrs.PlateCarree())
ax.set_extent((-20, 60, 40, 75))
ax.coastlines()
ax.add_feature(cfeature.BORDERS)
ax.add_feature(cfeature.LAKES)
ax.add_feature(cfeature.OCEAN)
ax.add_feature(cfeature.COASTLINE)
ax.add_feature(cfeature.RIVERS)
ax.legend(['Chernobyl', 'Radiosonde Stations'])

#ax.add_feature(provinces_50m)


ax.set_title('All Radiosonde Locations', fontsize=14)
plt.savefig('sonder.png')

In [10]:
filename = "/home/norah/master/Rs_data2/1986042600/52.45_31.0.csv"

data = pd.read_csv(filename)
#data = data[data['height'].notna()]
#data = data[data['windDirection'].notna()]
#data = data[data['windSpeed'].notna()]
data

,stationNumber,latitude,longitude,heightOfStation,pressure,nonCoordinateGeopotential,airTemperature,windDirection,windSpeed,height
0,41.0,52.45,31.0,124.0,100500.0,1220.0,295.2,100.0,5.0,124.362895
1,41.0,52.45,31.0,124.0,100000.0,1620.0,294.6,100.0,6.0,165.137615
2,41.0,52.45,31.0,124.0,92500.0,8170.0,289.0,115.0,6.0,832.823649
3,41.0,52.45,31.0,124.0,91400.0,NaN,NaN,115.0,6.0,NaN
4,41.0,52.45,31.0,124.0,89300.0,NaN,NaN,100.0,6.0,NaN
5,41.0,52.45,31.0,124.0,85000.0,15120.0,282.2,110.0,7.0,1541.284404
6,41.0,52.45,31.0,124.0,84800.0,15330.0,282.0,NaN,NaN,1562.691131
7,41.0,52.45,31.0,124.0,75800.0,NaN,NaN,115.0,11.0,NaN
8,41.0,52.45,31.0,124.0,73100.0,NaN,NaN,135.0,10.0,NaN
9,41.0,52.45,31.0,124.0,71900.0,28420.0,269.3,NaN,NaN,2897.043833


In [172]:
import os

for file in os.listdir("/home/norah/master/Rs_data2/1986050100/"):

    data = pd.read_csv("/home/norah/master/Rs_data2/1986050100/"+file)
    data = data[data['height'].notna()]
    limit= np.where(data["height"]>4000)

    num_nan = data["windDirection"][:limit[0][0]].isna().sum()
    frac_nan = num_nan/len(data["windDirection"][:limit[0][0]])

    if frac_nan<0.20:
        print(file)

56.38_30.6.csv
61.82_34.27.csv
63.7_9.6.csv
